In [1]:
import re
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import simplejson as json

/Users/azicon/anaconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
res = requests.get("https://piaofang.maoyan.com/dashboard/movie")
res.encoding = 'utf-8'
soup = bs(res.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="mobile" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="initial-scale=1, width=device-width, maximum-scale=1, user-scalable=no" name="viewport"/>
<link href="https://s3plus.meituan.net" rel="dns-prefetch"/>
<link href="https://p0.meituan.net" rel="dns-prefetch"/>
<link href="https://s3plus.meituan.net/v1/mss_e2821d7f0cfe4ac1bf9202ecf9590e67/cdn-prod/file:d6a62569/common.css" rel="stylesheet"/>
<style>
              html, body{
                  max-width: 430px;
                  margin: 0 auto;
              }
              .m-page{
                  display:flex;
                  align-items:center;
                  justify-content:center;
                  flex-direction: column;
                  min-height: 100vh;
              }
              .m-page .hint{
                  background: url(https://s0.meituan.net/bs/file/?f=myfe/piaofang:img/news/comments-none-ccd31fb5.png) no-repeat center .4rem;
                  background-size: 2rem;
 

In [3]:
browserOptions = Options()
#browserOptions.add_argument("--headless")

capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"
capa["goog:loggingPrefs"] = {"performance": "ALL"}
driver = webdriver.Chrome(desired_capabilities=capa)
wait = WebDriverWait(driver, 20)

In [4]:
#create snapshot of the entire page to prevent it from constantly changing
driver.get("https://piaofang.maoyan.com/dashboard/movie")
test = None
while not test:
    try:
        test = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'moviename-td')))
    except:
        driver.refresh();
        
logs_raw = driver.get_log("performance")
logs = [json.loads(lr["message"])["message"] for lr in logs_raw]

def log_filter(log_):
    return (
        # is an actual response
        log_["method"] == "Network.responseReceived"
        # and json
        and "json" in log_["params"]["response"]["mimeType"]
    )

responses = []

for log in filter(log_filter, logs):
    request_id = log["params"]["requestId"]
    resp_url = log["params"]["response"]["url"]
    print(f"Caught {resp_url}")
    response = driver.execute_cdp_cmd("Network.getResponseBody", {"requestId": request_id})
    print(response)
    responses.append(response)
    

Caught https://piaofang.maoyan.com/dashboard-ajax/movie?orderType=0&uuid=ab3e34cd-7030-4f17-b566-5bec383c4909&timeStamp=1644392378044&User-Agent=TW96aWxsYS81LjAgKE1hY2ludG9zaDsgSW50ZWwgTWFjIE9TIFggMTBfMTVfNykgQXBwbGVXZWJLaXQvNTM3LjM2IChLSFRNTCwgbGlrZSBHZWNrbykgQ2hyb21lLzk3LjAuNDY5Mi45OSBTYWZhcmkvNTM3LjM2&index=383&channelId=40009&sVersion=2&signKey=2ce5eed35a0eadd1db5bab664bcce44c
{'base64Encoded': False, 'body': '{"movieList":{"list":[{"avgSeatView":"6.2%","avgShowView":"10","boxRate":"32.9%","boxSplitUnit":{"num":"&#xf6cb;&#xeff1;&#xe823;&#xeff1;.&#xe293;&#xeb0f;","unit":"万"},"movieInfo":{"movieId":1446115,"movieName":"长津湖之水门桥","releaseInfo":"上映9天"},"showCount":120777,"showCountRate":"29.0%","splitBoxRate":"33.0%","splitBoxSplitUnit":{"num":"&#xf6cb;&#xf539;&#xf679;&#xea4c;.&#xf539;&#xe9eb;","unit":"万"},"sumBoxDesc":"29.20亿","sumSplitBoxDesc":"26.87亿"},{"avgSeatView":"6.6%","avgShowView":"9","boxRate":"24.4%","boxSplitUnit":{"num":"&#xeff1;&#xf679;&#xeb0f;&#xe9eb;.&#xeb0f;&#xeff1;","

In [5]:
body0 = json.loads(responses[0]['body'])
movieList = body0['movieList']['list']
date = body0['calendar']['today']

In [6]:
import pandas as pd
df = pd.DataFrame.from_records(movieList)
df

,avgSeatView,avgShowView,boxRate,boxSplitUnit,movieInfo,showCount,showCountRate,splitBoxRate,splitBoxSplitUnit,sumBoxDesc,sumSplitBoxDesc
0,6.2%,10,32.9%,{'num': '&#xf6cb;&#xeff1;&#xe823;&#xeff1;.&#xe...,"{'movieId': 1446115, 'movieName': '长津湖之水门桥', '...",120777,29.0%,33.0%,{'num': '&#xf6cb;&#xf539;&#xf679;&#xea4c;.&#xf...,29.20亿,26.87亿
1,6.6%,9,24.4%,{'num': '&#xeff1;&#xf679;&#xeb0f;&#xe9eb;.&#xe...,"{'movieId': 1433696, 'movieName': '这个杀手不太冷静', ...",103086,24.8%,24.3%,{'num': '&#xea4c;&#xeb0f;&#xe293;&#xf679;.&#xe...,16.80亿,15.32亿
2,8.0%,9,16.1%,{'num': '&#xe9eb;&#xe293;&#xe823;&#xf6cb;.&#xe...,"{'movieId': 1383307, 'movieName': '奇迹·笨小孩', 'r...",67612,16.2%,16.0%,{'num': '&#xe9eb;&#xeff1;&#xeb0f;&#xf8a5;.&#xf...,8.26亿,7.55亿
3,13.1%,14,14.7%,{'num': '&#xe9eb;&#xeff1;&#xf6cb;&#xe823;.&#xf...,"{'movieId': 1405863, 'movieName': '熊出没·重返地球', ...",41100,9.8%,14.8%,{'num': '&#xe9eb;&#xe9eb;&#xe823;&#xf8a5;.&#xf...,6.84亿,6.25亿
4,5.0%,6,6.2%,{'num': '&#xf539;&#xf679;&#xeff1;&#xf6cb;.&#xe...,"{'movieId': 1367251, 'movieName': '狙击手', 'rele...",45931,11.0%,6.2%,{'num': '&#xf8a5;&#xe293;&#xeff1;.&#xf539;&#xf...,3.37亿,3.09亿
5,9.9%,10,3.2%,{'num': '&#xf6cb;&#xea4c;&#xf6cb;.&#xe9eb;&#xe...,"{'movieId': 1445469, 'movieName': '喜羊羊与灰太狼之筐出未...",14069,3.3%,3.1%,{'num': '&#xeff1;&#xf8a5;&#xe9eb;.&#xe823;&#xf...,1.19亿,1.09亿
6,3.6%,4,1.5%,{'num': '&#xe9eb;&#xf6cb;&#xeff1;.&#xe293;&#xe...,"{'movieId': 1389048, 'movieName': '四海', 'relea...",17603,4.2%,1.5%,{'num': '&#xe9eb;&#xea4c;&#xeff1;.&#xf6cb;&#xe...,4.94亿,4.46亿
7,7.3%,8,0.2%,"{'num': '&#xea4c;&#xf6cb;.&#xf539;&#xeff1;', '...","{'movieId': 1328669, 'movieName': '汪汪队立大功大电影',...",1217,0.2%,0.2%,"{'num': '&#xea4c;&#xf539;.&#xf6cb;&#xf6cb;', '...",7778.5万,6881.0万
8,2.5%,3,0.1%,"{'num': '&#xea4c;&#xf679;.&#xe9eb;&#xe823;', '...","{'movieId': 1318977, 'movieName': '小虎墩大英雄', 'r...",2775,0.6%,0.1%,"{'num': '&#xe9eb;&#xeb0f;.&#xf8a5;&#xe9eb;', '...",1875.7万,1742.3万
9,10.0%,10,<0.1%,"{'num': '&#xf6cb;.&#xf679;&#xf679;', 'unit': '万'}","{'movieId': 1394182, 'movieName': '带你去见我妈', 'r...",136,<0.1%,<0.1%,"{'num': '&#xeff1;.&#xea4c;&#xe9eb;', 'unit': '万'}",2304.0万,2046.6万


In [7]:
integerLookup = {'&#xe984': 1, '&#xf43b': 2, '&#xf5fe': 3, '&#xea24': 4, '&#xef0d': 5, 
                 '&#xea11': 6, '&#xf3df': 7, '&#xec39': 8, '&#xea1b': 9, '&#xeffa': 0}

unitLookup = {'百': 100, '千': 1000, '万': 10000, '亿': 1*10**8}

#converts the weird character to a float
def convertToFloat(string):
    spCharLst = string.split(';')
    result = ''
    for i in spCharLst:
        if len(i) > 7: #has a dot in front
            result += '.' + str(integerLookup[i[1:]])
        elif len(i) == 7: #in case of bad parsing
            result += str(integerLookup[i])
    return float(result)

#helper function for converting the entire block to a single int
def convertDictToInt(dictionary):
    return int(convertToFloat(dictionary['num']) * unitLookup[dictionary['unit']])


In [9]:
convertDictToInt(df['boxSplitUnit'][0]) #裂开，不是bijection

KeyError: '&#xf6cb'

In [5]:
!pip install fontTools
from fontTools.ttLib import TTFont
headers = {
            "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) "
              "Chrome/66.0.3359.139 Safari/537.36 "
    }
index_url = 'http://maoyan.com/'
response_index = requests.get(index_url, headers=headers).text
woff_ = re.search(r"url\('(.*\.woff)'\)", response_index).group(1)
woff_url = 'http:' + woff_
response_woff = requests.get(woff_url, headers=headers).content

print(response_woff)
with open('fonts.woff', 'wb') as f:
    f.write(response_woff)
    
font1 = TTFont('fonts.woff')
font1.saveXML('fonts.xml')

     |████████████████████████████████| 895 kB 8.9 MB/s            
You should consider upgrading via the '/Users/azicon/anaconda3/bin/python -m pip install --upgrade pip' command.


2 extra bytes in post.stringData array


b'wOFF\x00\x01\x00\x00\x00\x00\t\x00\x00\x0b\x00\x00\x00\x00\x0c\xd0\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00GSUB\x00\x00\x01\x08\x00\x00\x003\x00\x00\x00B\xb0\xfe\xb3\xedOS/2\x00\x00\x01<\x00\x00\x00D\x00\x00\x00VV\xeeO\xabcmap\x00\x00\x01\x80\x00\x00\x00\xbb\x00\x00\x02L5\xf0\x9f\xa9glyf\x00\x00\x02<\x00\x00\x04r\x00\x00\x05L\xf0\xf3 \x11head\x00\x00\x06\xb0\x00\x00\x00/\x00\x00\x006\x1fr\x89\xf0hhea\x00\x00\x06\xe0\x00\x00\x00\x1c\x00\x00\x00$\x07\x8a\x039hmtx\x00\x00\x06\xfc\x00\x00\x00\x12\x00\x00\x000\x1a\x9e\x00\x00loca\x00\x00\x07\x10\x00\x00\x00\x1a\x00\x00\x00\x1a\x08l\x06\xc4maxp\x00\x00\x07,\x00\x00\x00\x1f\x00\x00\x00 \x01\x19\x00Hname\x00\x00\x07L\x00\x00\x01W\x00\x00\x02\x85\x90\x08h\x0bpost\x00\x00\x08\xa4\x00\x00\x00[\x00\x00\x00\x8f\xbcH\x06\xb1x\x9cc`d``\xe0b\x90c\xd0a`tq\xf3\ta\xe0``a\x80\x00\x90\x0ccNfz"P\x0c\xca\x03\xca\xb1\x80i\x0e f\x83\x88\x02\x00\x8a#\x03O\x00x\x9cc`d\xb2`\x9c\xc0\xc0\xca\xc0\xc1\xd4\xc9t\

In [8]:
df['boxSplitUnit'] = df['boxSplitUnit'].apply(convertDictToInt)
df['splitBoxSplitUnit'] = df['splitBoxSplitUnit'].apply(convertDictToInt)
df['movieInfo'] = df['movieInfo'].apply(lambda x : x['movieName'])
df

KeyError: '&#xf6cb'

In [ ]:
#大盘
body0['movieList']['nationBoxInfo']